# Importing packages

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm

# Relhum conversion functions

In [2]:
def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*np.polyval(a_ice,T-T0)))

def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = np.float32(T)
    qv = np.float32(qv)
    p = np.float32(p)
    
    return Rv*p*qv/(R*esat(T))

# Data processing functions

In [41]:
def do_month(month):
    datasets = !ls
    n = str(month)
    datasets = [x for x in datasets if "h1.0000-" + n.zfill(2) in x]
    return xr.open_mfdataset(datasets)

def make_nninput(spData, family, save_diagnostics = False, full_run = False):
    nntbp = np.float32(spData["NNTBP"].values)
    nnqbp = np.float32(spData["NNQBP"].values)
    p0 = np.float32(spData["P0"].values)
    ps = np.float32(spData["NNPS"].values)
    hyam = np.float32(spData["hyam"].values)
    hybm = np.float32(spData["hybm"].values)
    relhum = np.float32(spData["RELHUM"].values)
    tphystnd = np.float32(spData["TPHYSTND"].values)
    phq = np.float32(spData["PHQ"].values)

    p0 = np.float32(np.array(list(set(p0))))
    print("loaded in data")
    newhum = np.float32(np.zeros((spData["time"].shape[0],\
                                  spData["lev"].shape[0], \
                                  spData["lat"].shape[0], \
                                  spData["lon"].shape[0])))
    lats = spData["lat"]
    lons = spData["lon"]
    print("starting for loop")
    for i in tqdm(range(len(lats))):
        for j in range(len(lons)):
            latIndex = i
            lonIndex = j
            R = np.float32(287.0)
            Rv = np.float32(461.0)
            p = np.float32(p0 * hyam + ps[:, None, latIndex, lonIndex] * hybm) # Total pressure (Pa)
            T = np.float32(nntbp[:, :, latIndex, lonIndex])
            qv = np.float32(nnqbp[:, :, latIndex, lonIndex])
            p = np.float32(p)
            newhum[:,:, latIndex, lonIndex] = np.float32(Rv*p*qv/(R*esat(T)))
    
    nntbp = np.float32(np.moveaxis(nntbp[1:,:,:,:],0,1))
    print("nntbp")
    print(nntbp.shape)
    
    nnqbp = np.float32(np.moveaxis(nnqbp[1:,:,:,:],0,1))
    print("nnqbp")
    print(nnqbp.shape)
    
    lhflx = np.float32(spData["LHFLX"].values[np.newaxis,:-1,:,:])
    print("lhflx")
    print(lhflx.shape)
    
    shflx = np.float32(spData["SHFLX"].values[np.newaxis,:-1,:,:])
    print("shflx")
    print(shflx.shape)
    
    ps = np.float32(spData["NNPS"].values[np.newaxis,1:,:,:])
    print("ps")
    print(ps.shape)
    
    solin = np.float32(spData["SOLIN"].values[np.newaxis,1:,:,:])
    print("solin")
    print(solin.shape)
    
    newhum = np.float32(np.moveaxis(newhum[1:,:,:,:],0,1))
    print("newhum")
    print(newhum.shape)
    
    oldhum = np.float32(np.moveaxis(relhum[1:,:,:,:],0,1))
    print("oldhum")
    print(oldhum.shape)
    
    tphystnd = np.float32(np.moveaxis(tphystnd[1:,:,:,:],0,1))
    print("tphystnd")
    print(tphystnd.shape)
    
    phq = np.float32(np.moveaxis(phq[1:,:,:,:],0,1))
    print("phq")
    print(phq.shape)
        
    if family == "specific":
        nnInput = np.float32(np.concatenate((nntbp, \
                                             nnqbp, \
                                             ps, \
                                             solin, \
                                             shflx, \
                                             lhflx)))
        
        nnTarget = np.float32(np.concatenate((tphystnd, phq)))
    
    elif family == "relative":
        nnInput = np.float32(np.concatenate((nntbp, \
                                         newhum, \
                                         ps, \
                                         solin, \
                                         shflx, \
                                         lhflx)))
                             
        nnTarget = np.float32(np.concatenate((tphystnd, phq)))
    
    if full_run:
        nnInput = nnInput[:,:-1,:,:] #the last timestep of a run can have funky values
        nnTarget = nnTarget[:,:-1,:,:] #the last timestep of a run can have funky values
    
    print("nnInput")
    nnInput.shape
    
    print("nnTarget")
    nnTarget.shape
    
    errors = (newhum-oldhum/100).flatten()
    result = "Mean relative humidity conversion error: " + str(np.mean(errors)) + "\n"
    result = result + "Variance for relative humidity conversion error: " + str(np.var(errors)) + "\n"
    result = result + "nntbp.shape: " + str(nntbp.shape) + "\n"
    result = result + "nnqbp.shape: " + str(nnqbp.shape) + "\n"
    result = result + "lhflx.shape: " + str(lhflx.shape) + "\n"
    result = result + "shflx.shape: " + str(shflx.shape) + "\n"
    result = result + "ps.shape: " + str(ps.shape) + "\n"
    result = result + "solin.shape: " + str(solin.shape) + "\n"
    result = result + "newhum.shape: " + str(newhum.shape) + "\n"
    result = result + "oldhum.shape: " + str(oldhum.shape) + "\n"
    result = result + "tphystnd.shape: " + str(tphystnd.shape) + "\n"
    result = result + "phq.shape: " + str(phq.shape) + "\n"
    result = result + "nnInput.shape: " + str(nnInput.shape) + "\n"
    print(result)

    if save_diagnostics:
        diagnostics = 'diagnostics_' + str(month) + '.txt'
        with open(diagnostics, 'a') as fp:
            fp.write(result)
    
    return nnInput, nnTarget

def combine_arrays(*args, contiguous = True):
    if contiguous: # meaning each spData was part of the same run
        return np.concatenate((args), axis = 1)[:,:-1,:,:]
    return(np.concatenate((args), axis = 1))
                         
def sample_indices(size, spacing, fixed = True):
    numIndices = np.round(size/spacing)
    if fixed:
        indices = np.array([int(x) for x in np.round(np.linspace(1,size,int(numIndices)))])-1
    else:
        indices = list(range(size))
        np.random.shuffle(indices)
        indices = indices[0:int(numIndices)]
    return indices

def reshape_input(nnData, subsample = False, spacing = 5):
    if subsample:
        nnData = nnData[:,:,:,sampleIndices(nnData.shape[3], spacing, True)]
    nnData = nnData.ravel(order = 'F').reshape(64,-1,order = 'F')
    return nnData

def reshape_target(nnData, subsample = False, spacing = 5):
    if subsample:
        nnData = nnData[:,:,:,sampleIndices(nnData.shape[3], spacing, True)]
    nnData = nnData.ravel(order = 'F').reshape(60,-1,order = 'F')
    return nnData

def normalize_input(family, split = .2):
    
    # specific heat of air = 1004 J/ K / kg
    # latent heat of vaporization 2.5*10^6

    heatScale = 1004
    moistScale = 2.5e6
    
    if variation == 0:
        with open(path + 'nnDataSpecific_long_5.npy', 'rb') as f:
            arr = np.load(f)
    if variation == 1:
        with open(path + 'nnDataRelative_long_5.npy', 'rb') as f:
            arr = np.load(f)
    
    X = arr[0:64,:].transpose()
    y = arr[64:,:].transpose()
    
    print("X_train shape: ")
    print(X_train.shape)
    print("X_test shape: ")
    print(X_test.shape)
    print("y_train shape: ")
    print(y_train.shape)
    print("y_test shape: ")
    print(y_test.shape)

    train_mu = np.mean(X_train, axis = 0)[:, np.newaxis]
    train_std = np.std(X_train, axis = 0)[:, np.newaxis]
    train_min = X_train.min(axis = 0)[:, np.newaxis]
    train_max = X_train.max(axis = 0)[:, np.newaxis]
    
    if normalization == "standard":
        inpsub = train_mu
        inpdiv = train_std
        
    elif normalization == "range":
        inpsub = train_min
        inpdiv = train_max - train_min
        
    norms_input = [inpsub, inpdiv]
    
    print("INP_SUB shape: ")
    print(inpsub.shape)
    print("INP_DIV shape: ")
    print(inpdiv.shape)
    
    norm_xtrain = (X_train.transpose() - inpsub)/inpdiv
    norm_xtrain = norm_xtrain.transpose()
    norm_xtest = (X_test.transpose() - inpsub)/inpdiv
    norm_xtest = norm_xtest.transpose()

    
    print("norm X_train shape: ")
    print(norm_xtrain.shape)
    print("norm X_test shape: ")
    print(norm_xtest.shape)
    
    return [[norm_xtrain, norm_xtest], norms_input]

def normalize_target(variation, path, normalization, split = .2):
    
    # specific heat of air = 1004 J/ K / kg
    # latent heat of vaporization 2.5*10^6

    heatScale = 1004
    moistScale = 2.5e6
    
    if variation == 0:
        with open(path + 'nnDataSpecific_long_5.npy', 'rb') as f:
            arr = np.load(f)
    if variation == 1:
        with open(path + 'nnDataRelative_long_5.npy', 'rb') as f:
            arr = np.load(f)
    
    print("y_train shape: ")
    print(y_train.shape)
    print("y_test shape: ")
    print(y_test.shape)

    outscale = np.concatenate((np.repeat(heatScale, 30), np.repeat(moistScale, 30)))
    
    if normalization == "standard":
        inpsub = train_mu
        inpdiv = train_std
        
    elif normalization == "range":
        inpsub = train_min
        inpdiv = train_max - train_min
        
    norms_output = outscale
    
    print("INP_SUB shape: ")
    print(inpsub.shape)
    print("INP_DIV shape: ")
    print(inpdiv.shape)
    print("outscale shape: ")
    print(outscale.shape)
    
    norm_ytrain[:, 0:30] = norm_ytrain[:, 0:30]*outscale[0:30]
    norm_ytrain[:, 30:60] = norm_ytrain[:, 30:60]*outscale[30:60]
    
    norm_ytest[:, 0:30] = norm_ytest[:, 0:30]*outscale[0:30]
    norm_ytest[:, 30:60] = norm_ytest[:, 30:60]*outscale[30:60]
    
    print("norm y_train shape: ")
    print(norm_ytrain.shape)
    print("norm y_test shape: ")
    print(norm_ytest.shape)
    
    return [[norm_ytrain, norm_ytest], norms_output]

def prepare_ingredients32(variation, files, path = "./"):
    if variation == 0:
        path = path + "specific/"
    if variation == 1:
        path = path + "relative/"
    with open(path + "trainInput.npy", 'wb') as f:
        np.save(f, np.float32(files[1][0]))
    with open(path + "testInput.npy", 'wb') as f:
        np.save(f, np.float32(files[1][1]))
    with open(path + "trainOutput.npy", 'wb') as f:
        np.save(f, np.float32(files[1][2]))
    with open(path + "testOutput.npy", 'wb') as f:
        np.save(f, np.float32(files[1][3]))
    np.savetxt(path + "normalization/inp_sub.txt", np.float32(files[2][0]), delimiter=',')
    np.savetxt(path + "normalization/inp_div.txt", np.float32(files[2][1]), delimiter=',')
    np.savetxt(path + "normalization/out_scale.txt", np.float32(files[2][2]), delimiter=',')

In [35]:
nnInput_10, nnTarget_10 = make_nninput(do_month(10), "relative")
nnInput_11, nnTarget_11 = make_nninput(do_month(11), "relative")

unshaped_input = combine_arrays(nnInput_10, nnInput_11)
unshaped_target = combine_arrays(nnTarget_10, nnTarget_11)

reshaped_input = reshape_input(combine_arrays(nnInput_10, nnInput_11))
reshaped_target = reshape_target(combine_arrays(nnTarget_10, nnTarget_11))


loaded in data
starting for loop


100%|████████████████████████████████████████████████████████████████████████████| 64/64 [00:03<00:00, 18.06it/s]


nntbp
(30, 95, 64, 128)
nnqbp
(30, 95, 64, 128)
lhflx
(1, 95, 64, 128)
shflx
(1, 95, 64, 128)
ps
(1, 95, 64, 128)
solin
(1, 95, 64, 128)
newhum
(30, 95, 64, 128)
oldhum
(30, 95, 64, 128)
tphystnd
(30, 95, 64, 128)
phq
(30, 95, 64, 128)
nnInput
nnTarget
Mean relative humidity conversion error: 0.0043063145
Variance for relative humidity conversion error: 0.00047256332
nntbp.shape: (30, 95, 64, 128)
nnqbp.shape: (30, 95, 64, 128)
lhflx.shape: (1, 95, 64, 128)
shflx.shape: (1, 95, 64, 128)
ps.shape: (1, 95, 64, 128)
solin.shape: (1, 95, 64, 128)
newhum.shape: (30, 95, 64, 128)
oldhum.shape: (30, 95, 64, 128)
tphystnd.shape: (30, 95, 64, 128)
phq.shape: (30, 95, 64, 128)
nnInput.shape: (64, 95, 64, 128)

loaded in data
starting for loop


100%|████████████████████████████████████████████████████████████████████████████| 64/64 [00:03<00:00, 17.75it/s]


nntbp
(30, 95, 64, 128)
nnqbp
(30, 95, 64, 128)
lhflx
(1, 95, 64, 128)
shflx
(1, 95, 64, 128)
ps
(1, 95, 64, 128)
solin
(1, 95, 64, 128)
newhum
(30, 95, 64, 128)
oldhum
(30, 95, 64, 128)
tphystnd
(30, 95, 64, 128)
phq
(30, 95, 64, 128)
nnInput
nnTarget
Mean relative humidity conversion error: 0.0041737375
Variance for relative humidity conversion error: 0.00046795444
nntbp.shape: (30, 95, 64, 128)
nnqbp.shape: (30, 95, 64, 128)
lhflx.shape: (1, 95, 64, 128)
shflx.shape: (1, 95, 64, 128)
ps.shape: (1, 95, 64, 128)
solin.shape: (1, 95, 64, 128)
newhum.shape: (30, 95, 64, 128)
oldhum.shape: (30, 95, 64, 128)
tphystnd.shape: (30, 95, 64, 128)
phq.shape: (30, 95, 64, 128)
nnInput.shape: (64, 95, 64, 128)



In [36]:
unshaped_input.shape

(64, 189, 64, 128)

In [37]:
unshaped_target.shape

(60, 189, 64, 128)

In [38]:
reshaped_input.shape

(64, 1548288)

In [39]:
reshaped_target.shape

(60, 1548288)

In [40]:
189*64*128

1548288

(64, 190, 64, 128)

# run the code

In [ ]:
#saveNNInput(1)

In [ ]:
#saveNNInput(2)

In [ ]:
#saveNNInput(3)

In [ ]:
#saveNNInput(4)

In [ ]:
#saveNNInput(5)